## Prepare

In [1]:
# !kaggle competitions download -c dog-breed-identification

In [2]:
import os
import re
import zipfile
from collections import OrderedDict

import paddle
import pandas as pd
from bidict import bidict
from tqdm.notebook import tqdm

from utils import shared_code

In [3]:
train_labels_df = pd.read_csv('./data/labels.csv')
train_labels_df.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
targets_mapping = bidict(enumerate([
    "affenpinscher", "afghan_hound", "african_hunting_dog", "airedale", "american_staffordshire_terrier",
    "appenzeller", "australian_terrier", "basenji", "basset", "beagle", "bedlington_terrier",
    "bernese_mountain_dog", "black-and-tan_coonhound", "blenheim_spaniel", "bloodhound", "bluetick",
    "border_collie", "border_terrier", "borzoi", "boston_bull", "bouvier_des_flandres", "boxer",
    "brabancon_griffon", "briard", "brittany_spaniel", "bull_mastiff", "cairn", "cardigan",
    "chesapeake_bay_retriever", "chihuahua", "chow", "clumber", "cocker_spaniel", "collie",
    "curly-coated_retriever", "dandie_dinmont", "dhole", "dingo", "doberman", "english_foxhound",
    "english_setter", "english_springer", "entlebucher", "eskimo_dog", "flat-coated_retriever", "french_bulldog",
    "german_shepherd", "german_short-haired_pointer", "giant_schnauzer", "golden_retriever", "gordon_setter",
    "great_dane", "great_pyrenees", "greater_swiss_mountain_dog", "groenendael", "ibizan_hound", "irish_setter",
    "irish_terrier", "irish_water_spaniel", "irish_wolfhound", "italian_greyhound", "japanese_spaniel",
    "keeshond", "kelpie", "kerry_blue_terrier", "komondor", "kuvasz", "labrador_retriever", "lakeland_terrier",
    "leonberg", "lhasa", "malamute", "malinois", "maltese_dog", "mexican_hairless", "miniature_pinscher",
    "miniature_poodle", "miniature_schnauzer", "newfoundland", "norfolk_terrier", "norwegian_elkhound",
    "norwich_terrier", "old_english_sheepdog", "otterhound", "papillon", "pekinese", "pembroke", "pomeranian",
    "pug", "redbone", "rhodesian_ridgeback", "rottweiler", "saint_bernard", "saluki", "samoyed", "schipperke",
    "scotch_terrier", "scottish_deerhound", "sealyham_terrier", "shetland_sheepdog", "shih-tzu",
    "siberian_husky", "silky_terrier", "soft-coated_wheaten_terrier", "staffordshire_bullterrier",
    "standard_poodle", "standard_schnauzer", "sussex_spaniel", "tibetan_mastiff", "tibetan_terrier",
    "toy_poodle", "toy_terrier", "vizsla", "walker_hound", "weimaraner", "welsh_springer_spaniel",
    "west_highland_white_terrier", "whippet", "wire-haired_fox_terrier", "yorkshire_terrier"
]))

batch_size = 32

In [45]:
base_image_trans = paddle.vision.transforms.Compose([
    paddle.vision.transforms.Resize(size=256),
    paddle.vision.transforms.CenterCrop(size=224),
    paddle.vision.transforms.ToTensor(),
    paddle.vision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
extend_image_trans = paddle.vision.transforms.Compose([
    paddle.vision.transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    paddle.vision.transforms.RandomHorizontalFlip(),
    paddle.vision.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    paddle.vision.transforms.ToTensor(),
    paddle.vision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# train
class TrainDataset(paddle.io.Dataset):
    def __init__(self):
        super().__init__()
        self.records = pd.read_csv('./data/labels.csv').to_dict(orient='records')

    def __getitem__(self, idx):
        cur_id = self.records[idx]['id']
        cur_breed = self.records[idx]['breed']
        label = targets_mapping.inverse[cur_breed]
        image = paddle.vision.image_load(f'./data/train/{cur_id}.jpg', backend='cv2')
        image = extend_image_trans(image)
        return image, label

    def __len__(self):
        return len(self.records)


all_ds = TrainDataset()
all_ds_split = int(len(all_ds) * 0.9)
train_ds = paddle.io.Subset(all_ds, indices=range(0, all_ds_split))
test_ds = paddle.io.Subset(all_ds, indices=range(all_ds_split, len(all_ds)))
train_dl = paddle.io.DataLoader(dataset=train_ds, shuffle=True, batch_size=batch_size)
test_dl = paddle.io.DataLoader(dataset=test_ds, shuffle=False, batch_size=batch_size)


# test
class TestDataset(paddle.io.Dataset):
    def __init__(self):
        super().__init__()
        self.samples = []
        path = os.path.expanduser('./data/test')
        for root, _, fnames in sorted(os.walk(path, followlinks=True)):
            for fname in sorted(fnames):
                f = os.path.join(root, fname)
                self.samples.append(f)

    def __getitem__(self, idx):
        path = self.samples[idx]
        name = re.findall(r'(\w*)\.jpg', path)[0]
        sample = paddle.vision.image_load(path, backend='cv2')
        sample = base_image_trans(sample)
        return sample, name

    def __len__(self):
        return len(self.samples)


pred_dl = paddle.io.DataLoader(dataset=TestDataset(), shuffle=False, batch_size=batch_size)

## Model

In [52]:
net = paddle.nn.Sequential(
    paddle.vision.models.resnet101(pretrained=True),
    paddle.nn.ReLU(),
    paddle.nn.Linear(1000, 256),
    paddle.nn.ReLU(),
    paddle.nn.Linear(256, len(targets_mapping))
)
for p in net[0].parameters():
    p.stop_gradient = True

In [53]:
scheduler = paddle.optimizer.lr.StepDecay(learning_rate=0.001, step_size=1, gamma=0.9, verbose=True)
optimizer = paddle.optimizer.Momentum(learning_rate=scheduler, parameters=net.parameters())
# optimizer = paddle.optimizer.Adam(parameters=[i for i in net.parameters() if i.stop_gradient == False])
paddle.set_device('gpu')

tot_epoch = 10
for epoch in range(tot_epoch):
    def run(data_loader, is_train=True):
        if is_train:
            net.train()
        else:
            net.eval()

        pbar = tqdm(iterable=data_loader, colour='#1d3557' if is_train else '#457b9d')
        summer = shared_code.MultiSummer()
        for data, labels in pbar:
            images = data.astype(paddle.float32)
            labels = paddle.to_tensor(labels, dtype='int64').reshape((-1, 1))
            y_hat = net(images)
            loss = paddle.nn.functional.cross_entropy(y_hat, labels)
            acc = paddle.metric.accuracy(y_hat, labels)
            summer.put('loss', loss)
            summer.put('acc', acc)
            if is_train:
                loss.backward()
                optimizer.step()
                optimizer.clear_grad()
            pbar.set_description(f'Epoch {epoch + 1}/{tot_epoch} - '
                                 f'loss: {summer.get("loss")} '
                                 f'acc: {summer.get("acc")}')


    run(train_dl, is_train=True)
    run(test_dl, is_train=False)

Epoch 0: StepDecay set learning rate to 0.001.


  0%|          | 0/288 [00:00<?, ?it/s]

C:\Users\kun\AppData\Local\Programs\Python\Python39\lib\site-packages\paddle\nn\layer\norm.py:652: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/288 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

## Predict

In [56]:
net.eval()
submission = []
pbar = tqdm(iterable=pred_dl)
for data, names in pbar:
    images = data.astype(paddle.float32)
    y_hat = net(images)
    y_hat_soft = paddle.nn.functional.softmax(y_hat)

    for name, pred in zip(names, y_hat_soft.numpy()):
        cur = OrderedDict()
        cur['id'] = name
        for idx, score in enumerate(pred):
            cur[targets_mapping[idx]] = score
        submission.append(cur)

  0%|          | 0/324 [00:00<?, ?it/s]

In [57]:
pd.DataFrame(submission).to_csv('submission.csv', index=False)
with zipfile.ZipFile('submission.zip', mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write('submission.csv', arcname='submission.csv')